# **European journal publications BERTopic**

In [1]:
import torch

c:\Users\user\anaconda3\envs\bertopic_2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("Is CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(0)) 

Is CUDA available: True
CUDA device count: 1
Current device: 0
Device name: NVIDIA GeForce RTX 4070 Laptop GPU


In [3]:
import pandas as pd
import numpy as np
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.metrics import silhouette_score
import re

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text

In [5]:
import os
os.getcwd()

'c:\\Users\\user\\OneDrive - TUM\\1. 연구\\In Progress\\Sci-tech semantic proximity and regional competitiveness\\Data\\Queries\\Sci-Tech Similarity'

In [6]:
import pandas as pd
pub_publication = pd.read_csv('eu_pub_prox_publication.csv')
pub_publication.head()

C:\Users\user\AppData\Local\Temp\ipykernel_40432\2166497544.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  pub_publication = pd.read_csv('eu_pub_prox_publication.csv')


,pubid,pubyear,abstract,itemtitle,source
0,2.0,1992,\r\n<p>Applications to release genetically mod...,Environmental risks from the release of geneti...,MOLECULAR ECOLOGY
1,3.0,1992,\r\n<p>A vector system was developed employing...,The construction of recA-deficient Rhizobium m...,MOLECULAR ECOLOGY
2,4.0,1992,\r\n<p>The use of DNA fingerprinting to identi...,DNA fingerprinting in clonal organisms,MOLECULAR ECOLOGY
3,5.0,1992,\r\n<p>Mitochondrial DNA (mtDNA) from 25 blue ...,Unusual mitochondrial DNA polymorphism in two ...,MOLECULAR ECOLOGY
4,7.0,1992,\r\n<p>Males of the bushcricket Poecilimon vel...,Last-male sperm precedence in the bushcricket ...,MOLECULAR ECOLOGY


In [7]:
# check the number of unique pubid in each pubyear
pub_publication.groupby('pubyear')['pubid'].nunique()

pubyear
1980     48454
1981     52298
1982     56122
1983     62820
1984     67034
1985     71256
1986     73766
1987     77961
1988     83830
1989     88437
1990     94818
1991    103016
1992     73907
1993       986
1994      1023
1995       795
1996      1155
1997     34417
1998    271457
1999    277423
2000    281163
2001    283183
2002    284335
2003    291954
2004    301536
2005    312048
2006    328168
2007    349233
2008    364407
2009    372445
2010    386176
2011    453469
2012    472901
2013    496512
2014    500520
2015    514727
2016    530416
2017    540196
2018    551802
2019    569131
2020    621721
2021    677744
2022      8843
Name: pubid, dtype: int64

In [8]:
import pandas as pd
pub_institution = pd.read_csv('eu_pub_prox_institution.csv')
pub_institution.head()

C:\Users\user\AppData\Local\Temp\ipykernel_40432\1327335224.py:2: DtypeWarning: Columns (3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  pub_institution = pd.read_csv('eu_pub_prox_institution.csv')


,pubid,pubyear,institution_id_te,organization,country,eu_nuts_id
0,2.0,1992,81734973.0,UNIV YORK,ENGLAND,UKE21
1,2.0,1992,81889854.0,UNIVERSITY OF YORK - UK,ENGLAND,UKE21
2,3.0,1992,82583283.0,UNIVERSITY OF BIELEFELD,GERMANY,DEA41
3,3.0,1992,68681333.0,UNIV BIELEFELD,GERMANY,DEA41
4,4.0,1992,76491407.0,UNIV NOTTINGHAM,ENGLAND,UKF14


In [9]:
# check the number of unique pubid in each pubyear
pub_institution.groupby('pubyear')['pubid'].nunique()

pubyear
1980     48454
1981     52298
1982     56122
1983     62820
1984     67034
1985     71256
1986     73766
1987     77961
1988     83830
1989     88437
1990     94818
1991    103016
1992     73907
1993       986
1994      1023
1995       795
1996      1155
1997     34417
1998    271457
1999    277423
2000    281163
2001    283183
2002    284335
2003    291954
2004    301536
2005    312048
2006    328168
2007    349233
2008    364407
2009    372445
2010    386176
2011    453469
2012    472901
2013    496512
2014    500520
2015    514727
2016    530416
2017    540196
2018    551802
2019    569131
2020    621721
2021    677744
2022      8843
Name: pubid, dtype: int64

In [10]:
pub_institution = pub_institution[['pubid', 'eu_nuts_id']]
pub_institution = pub_institution.drop_duplicates()
pub_institution = pub_institution.dropna() 
pub_institution.head()

,pubid,eu_nuts_id
0,2.0,UKE21
2,3.0,DEA41
4,4.0,UKF14
8,5.0,FR714
14,7.0,DE252


In [11]:
# slice eu_nuts_id to 4 digits 
pub_institution['eu_nuts_id'] = pub_institution['eu_nuts_id'].str.slice(0, 4)
pub_institution

,pubid,eu_nuts_id
0,2.0,UKE2
2,3.0,DEA4
4,4.0,UKF1
8,5.0,FR71
14,7.0,DE25
...,...,...
16384009,10637056.0,BE10
16384011,10637061.0,FR10
16384012,10637061.0,FR71
16384014,10637061.0,DEA1


In [12]:
# merge publication and institution on pubid 
df = pd.merge(pub_publication, pub_institution, on='pubid', how = 'outer')
df.head()

,pubid,pubyear,abstract,itemtitle,source,eu_nuts_id
0,2.0,1992,\r\n<p>Applications to release genetically mod...,Environmental risks from the release of geneti...,MOLECULAR ECOLOGY,UKE2
1,3.0,1992,\r\n<p>A vector system was developed employing...,The construction of recA-deficient Rhizobium m...,MOLECULAR ECOLOGY,DEA4
2,4.0,1992,\r\n<p>The use of DNA fingerprinting to identi...,DNA fingerprinting in clonal organisms,MOLECULAR ECOLOGY,UKF1
3,5.0,1992,\r\n<p>Mitochondrial DNA (mtDNA) from 25 blue ...,Unusual mitochondrial DNA polymorphism in two ...,MOLECULAR ECOLOGY,FR71
4,7.0,1992,\r\n<p>Males of the bushcricket Poecilimon vel...,Last-male sperm precedence in the bushcricket ...,MOLECULAR ECOLOGY,DE25


In [13]:
# check the lowest and highest pubyear 
df['pubyear'].min(), df['pubyear'].max()

(1980, 2022)

In [14]:
# check how many unique pubid there are in year 2017
df[df['pubyear'] == 2016]['pubid'].nunique()

530416

In [15]:
# check the number of unique pubid in each year
df.groupby('pubyear')['pubid'].nunique()

pubyear
1980     48454
1981     52298
1982     56122
1983     62820
1984     67034
1985     71256
1986     73766
1987     77961
1988     83830
1989     88437
1990     94818
1991    103016
1992     73907
1993       986
1994      1023
1995       795
1996      1155
1997     34417
1998    271457
1999    277423
2000    281163
2001    283183
2002    284335
2003    291954
2004    301536
2005    312048
2006    328168
2007    349233
2008    364407
2009    372445
2010    386176
2011    453469
2012    472901
2013    496512
2014    500520
2015    514727
2016    530416
2017    540196
2018    551802
2019    569131
2020    621721
2021    677744
2022      8843
Name: pubid, dtype: int64

In [16]:
# slice the data to only include data from 1980 to 2021
df = df[(df['pubyear'] >= 1980) & (df['pubyear'] <= 2021)]
df['pubyear'].unique()

array([1992, 1996, 1997, 1998, 1999, 2000, 2001, 1995, 2002, 2003, 2004,
       2006, 1994, 2005, 1989, 1986, 1993, 1990, 1987, 1988, 1991, 1985,
       2007, 1981, 2008, 1982, 1984, 1983, 2009, 1980, 2010, 2012, 2011,
       2013, 2015, 2014, 2017, 2016, 2018, 2019, 2020, 2021], dtype=int64)

In [17]:
# create 'period' column 
def assign_period(pubyear):
    if 1981 <= pubyear <= 1985:
        return 1
    elif 1986 <= pubyear <= 1990:
        return 2 
    elif 1991 <= pubyear <= 1995:
        return 3 
    elif 1996 <= pubyear <= 2000:
        return 4
    elif 2001 <= pubyear <= 2005:
        return 5
    elif 2006 <= pubyear <= 2010:
        return 6
    elif 2011 <= pubyear <= 2015:
        return 7
    elif 2016 <= pubyear <= 2020:
        return 8
    else:
        return 0 
    
df['period'] = df['pubyear'].apply(assign_period)
df.head()

,pubid,pubyear,abstract,itemtitle,source,eu_nuts_id,period
0,2.0,1992,\r\n<p>Applications to release genetically mod...,Environmental risks from the release of geneti...,MOLECULAR ECOLOGY,UKE2,3
1,3.0,1992,\r\n<p>A vector system was developed employing...,The construction of recA-deficient Rhizobium m...,MOLECULAR ECOLOGY,DEA4,3
2,4.0,1992,\r\n<p>The use of DNA fingerprinting to identi...,DNA fingerprinting in clonal organisms,MOLECULAR ECOLOGY,UKF1,3
3,5.0,1992,\r\n<p>Mitochondrial DNA (mtDNA) from 25 blue ...,Unusual mitochondrial DNA polymorphism in two ...,MOLECULAR ECOLOGY,FR71,3
4,7.0,1992,\r\n<p>Males of the bushcricket Poecilimon vel...,Last-male sperm precedence in the bushcricket ...,MOLECULAR ECOLOGY,DE25,3


In [18]:
# remove '\r\n<p>' from the abstracts 
df['abstract'] = df['abstract'].str.replace('\r\n<p>', '')
df.head()

,pubid,pubyear,abstract,itemtitle,source,eu_nuts_id,period
0,2.0,1992,Applications to release genetically modified o...,Environmental risks from the release of geneti...,MOLECULAR ECOLOGY,UKE2,3
1,3.0,1992,A vector system was developed employing the re...,The construction of recA-deficient Rhizobium m...,MOLECULAR ECOLOGY,DEA4,3
2,4.0,1992,The use of DNA fingerprinting to identify memb...,DNA fingerprinting in clonal organisms,MOLECULAR ECOLOGY,UKF1,3
3,5.0,1992,Mitochondrial DNA (mtDNA) from 25 blue tits Pa...,Unusual mitochondrial DNA polymorphism in two ...,MOLECULAR ECOLOGY,FR71,3
4,7.0,1992,Males of the bushcricket Poecilimon veluchianu...,Last-male sperm precedence in the bushcricket ...,MOLECULAR ECOLOGY,DE25,3


# **특정 기준 이상 논문 보유한 지역, 기간 선정**

In [19]:
# group by eu_nuts_id and period and count the number of unique pubid 
df_grouped = df.groupby(['eu_nuts_id', 'period'])['pubid'].nunique().reset_index()
df_grouped.head()

,eu_nuts_id,period,pubid
0,AT11,4,34
1,AT11,5,64
2,AT11,6,2
3,AT11,7,4
4,AT12,2,2


In [20]:
df_grouped.describe()

,period,pubid
count,1792.000000,1792.000000
mean,4.308036,1993.296875
std,1.994404,5352.149497
min,0.000000,1.000000
25%,3.000000,9.750000
50%,4.000000,99.000000
75%,6.000000,1167.750000
max,8.000000,94877.000000


In [21]:
# drop rows where pubid is less than 50 
df_grouped = df_grouped[df_grouped['pubid'] >= 100]
df_grouped

,eu_nuts_id,period,pubid
6,AT12,4,597
7,AT12,5,1309
8,AT12,6,116
13,AT13,4,11735
14,AT13,5,21362
...,...,...,...
1782,UKM6,5,661
1788,UKN0,4,3851
1789,UKN0,5,6627
1790,UKN0,6,575


In [22]:
df_grouped['eu_nuts_id'].nunique()

281

In [23]:
df_grouped.to_csv('pub_above_100.csv', index=False)

In [24]:
# drop rows in df where each pair of eu_nuts_id and period is not in df_grouped
df = df.merge(df_grouped[['eu_nuts_id', 'period']], on=['eu_nuts_id', 'period'], how='inner')
df

,pubid,pubyear,abstract,itemtitle,source,eu_nuts_id,period
0,5.0,1992,Mitochondrial DNA (mtDNA) from 25 blue tits Pa...,Unusual mitochondrial DNA polymorphism in two ...,MOLECULAR ECOLOGY,FR71,3
1,3798739.0,1995,A number of surface modification methods using...,Adhesion science and surface analysis. Typical...,JOURNAL OF ADHESION,FR71,3
2,3798867.0,1995,This paper describes the prototype of an exper...,An expert system for cement kilns analysis usi...,CHEMICAL ENGINEERING COMMUNICATIONS,FR71,3
3,3813535.0,1995,"Since the work of Pauwels, the forces exerted ...",Biomechanics of the hip: forces exerted during...,SURGICAL AND RADIOLOGIC ANATOMY,FR71,3
4,7083096.0,1993,"Throughout manufacturing history, management h...",Direct value added and economic weight factors...,INTERNATIONAL JOURNAL OF ADVANCED MANUFACTURIN...,FR71,3
...,...,...,...,...,...,...,...
3690762,7676856.0,2015,This paper provides recommendations of one nam...,Recommended names for pleomorphic genera in Do...,IMA FUNGUS,DEE0,7
3690763,7684153.0,2015,Background: In pediatric Hodgkin's lymphoma (p...,FDG-PET Response Prediction in Pediatric Hodgk...,CANCERS,DEE0,7
3690764,7690432.0,2015,"Background: In this work, two-dimensional (2D)...",Evaluation of 2D spatially selective MR spectr...,QUANTITATIVE IMAGING IN MEDICINE AND SURGERY,DEE0,7
3690765,7698533.0,2015,Feedback is generally accepted as a suitable i...,An analysis of smart metering information syst...,ENERGY RESEARCH & SOCIAL SCIENCE,DEE0,7


In [25]:
# customize stop words list
custom_stop_words = ["determining", "method", "includes", "based", "second", "using", 'showed', 'problem', "study", "non"
                     "12", "14", "15", "2014", "iii", "ii", "10", "1a", "11", "cm", "60", '28', '250', '17',
                     "25", "40", "model", "2005", "ar", "rh", "kv", "sb", "ac", "www", "nm", 'srr', 'xy', 'sigma',
                     "mo", "nm", "18", "ee", "pb", "gev", "mu", "rh", "ln", "a15", "13", "ag", '55', 'effect','nd3',
                     "nm", "qds", "iv", "ag", "rms", "le", "p53", "pl", "fe", "ir", "4abn", "nmr", "ni", '05', '74',
                     "ta", "tio2"< "thz", "db", "hz", "iso", "pss", "1310", "nu", "tf", "tf", '33', '350',
                     "km", "tev", "gev", "fb", "tl", "t2dm", "nlc", "dot", "si", "cb7", "mc", "soi", '100', 
                     "dots", "pi", "qd", "la", "cdw", "iqp", "ccl4", "no2", "mm", "p1", "bi", "ho", "rs", '00',
                     "qh", "tio2", "nu", "nir", "time", "bm12", "ssi", "u6", "current", "nmr", 'hs', 'sp',
                     "gev", "tot", "1h", "hh", "hi", "qds", "pss", "no2", "zb", "wz", "al", "ge", 'ff', 'der',
                     "ect", "mdd", "95", "girls", "uc", "cpe", "di", "uc", "er3", "new", 's190', 's208', '44',
                     "od", "ch3", "yb", "eu", "tb", "nr", "ope3", "4tp", "mu", "center", "la", '3b', 'eag1',
                     "number", "su", "ch4", "fe", "cr", "ni", "eat", "ws", "spp", "mn2", "mm", 'ss', 'oc',
                     "rf", "li", "2016", "16", "sic", "si", "von", "sept3", "0d", "36", 'igf1r', 'ma', 'mnc', ''
                     'o3', 'correctly', 'tb', 'te', 'il', 'cd39', 'cd73', 'au', 'ou', 'ml', 'mg', 'puo2', '298',
                     'sec', 'eta', 'ba122', 'al', 'cgm', 'h2s', 'ir', 'sni', '45', '75', 'tio2', '60', 'sp', '4000', 
                     'ge', 'si', '5p', '120', '2d', 'random', 'theory', 'li', 'qds', 'qd', 'p3ht', 'mu', '69', '71',
                     'p2', 'gw', '511', '3885', 'yb3', '25', 'sir', '240', 'ro', 't34', '10', '6p', 'bbb', 'c1', 'c3',
                     'nno', 'nlo', 'ag', 'sb', 'naf', 'ga', 'se', 'cu', 'gd', 'pf6', 'dots', 'dat', '4f', 
                     'ch3', 'center', 'centre', 'mc3t3', 'e1', '1064', 'jc', 'ga', 'cf3', 'bu', 'zn', 'near', 
                     'np', 'zn', 'cp', 'fr', 'nmc', 'nca', 'oco', 'dsb', 'ssb', 'oe', 'mn', '300', '80', 'b12x12',
                     'nh', 'br', 'mw', '77', 'ig', '24', 'sigma', '700', 'ij', '1h', 'approach', '4b', '343', 
                     's6', 'ce3', 'advantages', 'phase', '001', 'ad', 'dy3', '111', '2000', 'matter', '99',
                     'results', 'used', 'methods', 'analysis', 'used', 'associated', 'different', 'argue', 'article', 
                     'let', 'research', 'data', 'paper', 'examines', 'prove', 'proposed', 'samples', 'study', 'similar', 
                     'compared', 'significantly', 'high', 'publication', 'significant', 'use', 'studies', 'category', 
                     'use', 'field', 'group', 'years', '000', '2024', 'field', 'risk', 'non', 'age'
                     ]  

# 기본 영어 stop words (서술어, 조사 등) 목록 가져오기
default_stop_words = text.ENGLISH_STOP_WORDS

# 기본 영어 stop words에 추가할 단어들 합치기
all_stop_words = list(default_stop_words.union(custom_stop_words))

In [26]:
df.head()

,pubid,pubyear,abstract,itemtitle,source,eu_nuts_id,period
0,5.0,1992,Mitochondrial DNA (mtDNA) from 25 blue tits Pa...,Unusual mitochondrial DNA polymorphism in two ...,MOLECULAR ECOLOGY,FR71,3
1,3798739.0,1995,A number of surface modification methods using...,Adhesion science and surface analysis. Typical...,JOURNAL OF ADHESION,FR71,3
2,3798867.0,1995,This paper describes the prototype of an exper...,An expert system for cement kilns analysis usi...,CHEMICAL ENGINEERING COMMUNICATIONS,FR71,3
3,3813535.0,1995,"Since the work of Pauwels, the forces exerted ...",Biomechanics of the hip: forces exerted during...,SURGICAL AND RADIOLOGIC ANATOMY,FR71,3
4,7083096.0,1993,"Throughout manufacturing history, management h...",Direct value added and economic weight factors...,INTERNATIONAL JOURNAL OF ADVANCED MANUFACTURIN...,FR71,3


In [45]:
import pandas as pd
from tqdm import tqdm
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
from bertopic.vectorizers import ClassTfidfTransformer

# Create an empty list to store the results
all_results = []

# Iterate over each 'eu_nuts_id' and 'period', showing progress with tqdm
for nuts in tqdm(df['eu_nuts_id'].unique(), desc="NUTS regions"):
    for i in range(1, 9):
        # Create a filtered copy of the dataset for 'eu_nuts_id' and 'period'
        filtered_df = df[(df['eu_nuts_id'] == nuts) & (df['period'] == i)].copy()

        # Extract the 'abstract' column for BERTopic modeling, and ensure all entries are strings
        docs = filtered_df['abstract'].dropna().astype(str).tolist()

        if len(docs) == 0:
            print(f"No valid documents for NUTS region {nuts} and period {i}")
            continue

        try:
            # Term Frequency, Inverse Document Frequency & transform docs into vectors
            vectorizer_model = CountVectorizer(stop_words=all_stop_words)
            ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
            embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
            embeddings = embedding_model.encode(docs, show_progress_bar=True)

            # Dynamically set the number of neighbors for UMAP based on the number of documents
            n_neighbors = min(30, len(docs) - 1)  # Ensures n_neighbors <= number of documents 15 or 30
            if n_neighbors <= 2:  # UMAP requires n_neighbors to be at least 2
                print(f"Skipping NUTS region {nuts} and period {i} due to insufficient documents.")
                continue

            umap_model = UMAP(n_neighbors=n_neighbors, min_dist=0.1, metric='cosine') # 0.0-0.3, a lower min_dist makes the clusters tighgter, whereas a higher value makes dispersed clusters

            # hyper parameter 설정, 최대 10개의 주제를, 최소 min_topic_sizes 보다 큰 클러스터로 생성 
            n_gram_ranges = (1, 2)  # n-gram 범위 (1, 2, 3, 4-gram)
            min_topic_sizes = max(2, int(len(docs) * 0.01))  # 한 클러스터에 포함되는 최소 문서 수 (1, 1.5, 2, 3%)
            nr_topics_options = 30  # 최대 클러스터링 수 (5, 10, 15, 20개)

            # 문서별로 할당된 주제 topics와 확률 probs
            topic_model = BERTopic(
                n_gram_range=n_gram_ranges,
                min_topic_size=min_topic_sizes,
                nr_topics=nr_topics_options,
                embedding_model=embedding_model,
                vectorizer_model=vectorizer_model,
                calculate_probabilities=True,
                ctfidf_model=ctfidf_model,
                umap_model=umap_model
            )

            topics, probs = topic_model.fit_transform(docs, embeddings)

            # 결과를 저장
            freq = topic_model.get_topic_info()

            # Add 'eu_nuts_id' and 'period' columns to the topic info DataFrame
            freq['eu_nuts_id'] = nuts
            freq['period'] = i

            # Append the result to the list
            all_results.append(freq)

            print(f"Processed topic information for NUTS region {nuts} and period {i}")

        except Exception as e:
            print(f"Error processing NUTS region {nuts} and period {i}: {e}")

# Combine all the results into a single DataFrame
final_results = pd.concat(all_results, ignore_index=True)

# Save the final combined DataFrame to a single CSV file
final_results.to_csv('pub_bertopic.csv', index=False)

NUTS regions:   0%|          | 0/281 [00:00<?, ?it/s]

No valid documents for NUTS region FR71 and period 1
No valid documents for NUTS region FR71 and period 2


Batches: 100%|██████████| 4/4 [00:00<00:00,  7.10it/s]


Processed topic information for NUTS region FR71 and period 3


Batches: 100%|██████████| 680/680 [02:54<00:00,  3.90it/s]


...

No valid documents for NUTS region BE34 and period 7
No valid documents for NUTS region BE34 and period 8
No valid documents for NUTS region EL65 and period 1
No valid documents for NUTS region EL65 and period 2
No valid documents for NUTS region EL65 and period 3
No valid documents for NUTS region EL65 and period 4


Batches: 100%|██████████| 5/5 [00:05<00:00,  1.16s/it]


Processed topic information for NUTS region EL65 and period 5
No valid documents for NUTS region EL65 and period 6


NUTS regions: 100%|██████████| 281/281 [4:37:57<00:00, 59.35s/it]

No valid documents for NUTS region EL65 and period 7
No valid documents for NUTS region EL65 and period 8


In [1]:
import pandas as pd
final_results = pd.read_csv('eu_pub_bertopic_nuts2.csv')
final_results.head()

,Topic,Count,Name,Representation,Representative_Docs,nuts,period
0,-1,1,-1_sources_cw_photodetectors_various,"['sources', 'cw', 'photodetectors', 'various',...",['A wide range of photodetectors has been inve...,FR71,3
1,0,24,0_ice_lake_species_females,"['ice', 'lake', 'species', 'females', 'develop...",['The results of preliminary experiments in th...,FR71,3
2,1,18,1_mixtures_performance_computational_control,"['mixtures', 'performance', 'computational', '...",['We present some computational methods in a p...,FR71,3
3,2,10,2_periodic_cells_dimensional_chaos,"['periodic', 'cells', 'dimensional', 'chaos', ...",['The intermolecular pair potential responsibl...,FR71,3
4,3,6,3_plasma_axion_neutrinos_stars,"['plasma', 'axion', 'neutrinos', 'stars', 'ecr...","['Using thermal field theory methods, we recal...",FR71,3


In [2]:
final_results[final_results['Topic'] == -1]

,Topic,Count,Name,Representation,Representative_Docs,nuts,period
0,-1,1,-1_sources_cw_photodetectors_various,"['sources', 'cw', 'photodetectors', 'various',...",['A wide range of photodetectors has been inve...,FR71,3
8,-1,6368,-1_patients_species_elsevier_observed,"['patients', 'species', 'elsevier', 'observed'...","[""Multiple-year time series of the weight rati...",FR71,4
13,-1,1331,-1_hpv_star_algebra_mass,"['hpv', 'star', 'algebra', 'mass', 'observatio...",['We present a survey of the formaldehyde emis...,FR71,5
16,-1,192,-1_bone_gt_fracture_fractures,"['bone', 'gt', 'fracture', 'fractures', 'verte...","[""In the framework of the French V/HTR fuel de...",FR71,6
19,-1,97,-1_pax_pnh_core_aa,"['pax', 'pnh', 'core', 'aa', 'information', 'a...",['Background/Aims: Urinary albumin excretion i...,FR71,7
...,...,...,...,...,...,...,...
7563,-1,27,-1_sun_mutations_prpc_anal,"['sun', 'mutations', 'prpc', 'anal', 'conforma...",['Host-parasite relationships and pathogenicit...,MT00,5
7583,-1,55,-1_ha_caviar_process_sturgeon,"['ha', 'caviar', 'process', 'sturgeon', 'culti...",['Immature zygotic embryos from the American f...,RO22,5
7603,-1,11,-1_irrigation_indices_cloud_network,"['irrigation', 'indices', 'cloud', 'network', ...",['Irrigation management calls for objective cr...,EL53,5
7619,-1,9,-1_responders_weight_hazelnut_embryos,"['responders', 'weight', 'hazelnut', 'embryos'...","['Patients with schizophrenia (Sch), schizoaff...",BE34,5


In [4]:
# Step 1: Total count per eu_nuts_id and period
total_count = final_results.groupby(['nuts', 'period'])['Count'].sum()

# Step 2: Count for Topic == -1 per eu_nuts_id and period
minus_one_count = final_results[final_results['Topic'] == -1].groupby(['nuts', 'period'])['Count'].sum()

# Step 3: Calculate ratio
ratio = (minus_one_count / total_count).reset_index(name='topic_minus_one_ratio')

# Optional: rename columns for clarity
ratio = ratio.rename(columns={'nuts': 'NUTS', 'period': 'Period'})
ratio

,NUTS,Period,topic_minus_one_ratio
0,AT12,4,0.340678
1,AT12,5,0.022013
2,AT12,6,0.026549
3,AT13,4,0.206965
4,AT13,5,0.261160
...,...,...,...
891,UKM6,5,0.004777
892,UKN0,4,0.043901
893,UKN0,5,0.001894
894,UKN0,6,0.266332


In [5]:
ratio['topic_minus_one_ratio'].describe()

count    806.000000
mean       0.143991
std        0.102291
min        0.000411
25%        0.059674
50%        0.133577
75%        0.209203
max        0.503406
Name: topic_minus_one_ratio, dtype: float64

# **ChatGPT를 활용한 Label 생성**

In [6]:
import pandas as pd 
final_results = pd.read_csv('eu_pub_bertopic_nuts2.csv')

In [ ]:
from openai import OpenAI

# client = OpenAI(api_key="")

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are an AI assistant that generates labels for topics based on provided keywords."},
        {"role": "user", "content": '''['sources', 'cw', 'photodetectors', 'various', 'realized', '3d', 'pulsed', 'optimized', 'theoretical', 'wavelengths']
The list of words are the outcome of the topic modeling from science and technology related journal articles.
Generate a new label that best describes the topic of the articles as follow: <new science topic>
Provide a specific and detailed label. I want just a single label. I do not want descriptions for the labels.'''}
    ]
)

print(response.choices[0].message.content)

Advanced Photodetection and Wavelength Optimization in 3D Light Sources


In [ ]:
from openai import OpenAI

# client = OpenAI(api_key="")

final_results = pd.read_csv('eu_pub_bertopic_nuts2.csv')

# List to hold results 
results = []

# Iterate through each row in the DataFrame
for index, row in final_results.iterrows():
    keywords = row['Representation']

    # Prompt for OpenAI
    prompt = f'''{keywords}
    The list of words are the outcome of the topic modeling from science and technology related journal articles.
    Generate a new label that best describes the topic of the articles as follow: <new science topic>
    Provide a specific and detailed label. I want just a single label. I do not want descriptions for the labels.'''

    try:
        # Call the OpenAI API to generate the new label
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are an AI assistant that generates labels for topics based on provided keywords."},
                {"role": "user", "content": prompt}
            ]
        )

        # Get the label from the response 
        new_label = response.choices[0].message.content.strip()

        # Append the result to the list
        results.append({
            'nuts' : row['nuts'],
            'period' : row['period'],
            'keyword' : keywords,
            'content' : new_label
        })
    
    except Exception as e:
        print(f"Error processing row {index}: {e}")

# Convert the results list to a DataFrame
results_df = pd.DataFrame(results)

# Save the results to a CSV file
results_df.to_csv('eu_pub_bertopic_nuts2_labels.csv', index=False)